In [3]:
# Paruošia pūpsnių indeksus ML mokymui ir vertinimui.
# 
# Panaudojant pacientų įrašų vardų SubjCode sąrašus train_lst, validate_lst, test_lst iš disko,
# sukūriami atitinkami indeksų failai train_ind_lst, validate_ind_lst, test_ind_lst ir įrašomi į diską.
# Kartu su duomenimis ir all_beats_attr, jie naudojami klasifikatorių kūrimui. 
# 
# Nebalansuotas variantas.
# 
# Paruošti pūpsnių indeksų masyvai gali būti panaudoti tiesioginiam sekų 
# formavimui iš EKG įrašų (rec) skaičiavimo metu arba sekų Numpy masyvų formavimui
#  
# //////////////////////////////////////////////////////////////////////////////////////////////////////
#
import pandas as pd
import numpy as np
from numpy import loadtxt, savetxt
import matplotlib.pyplot as plt
from pathlib import Path
from icecream import ic
import json, time, sys

from zive_util_vu import create_dir
from zive_util_vu import anotaciju_pasiskirstymas_v2, runtime
from zive_util_vu import get_annotations_table, print_annotations_table


my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE\VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'DUOM_VU'

# Aplankas su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_folder = 'records_npy'

# Duomenų schemos aplanko pavadinimas
SCHEMA_FOLDER = 'schema_NSV_50_nbal'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Anotacijoms priskirtos klasės
selected_beats = {'N':0, 'S':1, 'V':2, 'U':3}

# Sekos kanalų skaičius
n_channels = 1



# Train imties pacientai
# DS1 = [ 10000, 10001]

# Train imties pacientai
# DS1 = [101, 106, 108, 109, 112, 114, 115, 116, 118, 119, 122, 124, 201, 203, 205, 208, 209, 215, 220, 223,230]

# Validate imties pacientai
# DS2 = [10010, 10031, 10032, 10060, 10061]

# Validate imties pacientai
# DS2 = [100, 103, 105, 111, 113, 117, 121, 123, 200, 202, 210, 212, 213, 214, 219, 221, 222, 228, 231, 232, 233, 234]

# Kiekis pūpsnių, kurie atmetami EKG įrašo pradžioje ir pabaigoje
beats_skiped = 10

# ///////////////////////////////////////////////////////////////////////////////

# Anotacijų sąrašas
annot_list = [key for key in selected_beats]

# swapped_selected_beats
swapped_selected_beats = {v: k for k, v in selected_beats.items()}

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuskaitome ir suformuojame DS2
file_path = Path(rec_dir, 'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)
DS2 = info_create['SubjCodes']
print(DS2)


# Nuoroda į duomenų schemos aplanką ir vertinimo imties sekų indeksų failas
SCHEMA_PATH = Path(db_path, SCHEMA_FOLDER)
all_beats_attr_fname = 'all_beats_attr_z.csv'

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas

# Klasių skaičius
n_classes = len(selected_beats)

print("\nParuošia EKG pūpsnių indeksus ML mokymui ir vertinimui")
print("Nebalansuotas variantas")

print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", db_folder)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų sąrašas:\n",SubjCodes)
# print("Pacientų sąrašas mokymui:", DS1)
# print("Pacientų sąrašas vertinimui:", DS2)

print("\nDuomenų schemos aplankas: ", SCHEMA_PATH)
print('Klasifikavimo schema:', selected_beats)
print('Klasių skaičius:', len(selected_beats))
print('Pūpsnių atributų masyvo failas:', all_beats_attr_fname)

print("\nParametrai pūpsnių kiekiui apriboti")
print(f"beats_skiped: {beats_skiped}")
print(f"Kiekviename EKG įraše praleisti {beats_skiped} pirmi ir {beats_skiped} paskutiniai pūpsniai")



OS in my system :  win32
[10000, 10001, 10002, 10003, 10010, 10020, 10021, 10022, 10023, 10024, 10025, 10030, 10031, 10032, 10033, 10034, 10035, 10036, 10040, 10050, 10051, 10052, 10053, 10054, 10055, 10056, 10057, 10058, 10059, 10060, 10061, 10062, 10063, 10064, 10065, 10066, 10067, 10068, 10069, 10070, 10071, 10072, 10080, 10081, 10082, 10083, 10084, 10085, 10086, 10087, 10088, 10089, 10090, 10100, 10101, 10102, 10103, 10104, 10105, 10106, 10107, 10108, 10109, 10110, 10120, 10121, 10122, 10123, 10124, 10125, 10126, 10127, 10128, 10129, 10130, 10131, 10132, 10133, 10134, 10140, 10141, 10142, 10143, 10144, 10145, 10146, 10147, 10148, 10149, 10150, 10151, 10152, 10153, 10154, 10155, 10156, 10157, 10158, 10159, 10160, 10161, 10162, 10163, 10164, 10165, 10166, 10167, 10168, 10169, 10170, 10171, 10172, 10173, 10174, 10175, 10176, 10180, 10190, 10191, 10192, 10193, 10194, 10195, 10196, 10197, 10198, 10199, 10200, 10201, 10210, 10211, 10212, 10213, 10214, 10215, 10216, 10217, 10220, 10221, 1

In [5]:

pd.set_option("display.max_rows", 1000, "display.max_columns", 19)
pd.set_option('display.width', 1000)

cols_pattern = ['N', 'S', 'V']
 
#  from mit2zive_util import create_set_from_rec
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from zive_util import split_SubjCode

start_time = time.time()

# Nuskaitome pūpsnių atributų freimą iš rec_dir aplanko
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr = pd.read_csv(file_path, index_col=0)

# Visų sekų indeksų sąrašas
all_beat_indices = all_beats_attr.index
# print(all_beat_indices)
# DS = DS1 + DS2
# all_ind_lst = []
# for userNr in DS:
#     selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
#     all_ind_lst.extend(selected_ind.to_list())

print("\nAnotacijų pasiskirstymas visuose duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Suformuojame sekų masyvus klasifikatoriaus mokymui ir vertinimui

# MOKYMO IMTIS

# Suformuojame mokymo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus
# train_ind_lst = []
# for userNr in DS1:
#     selected_ind = all_beat_indices[all_beats_attr['userNr']==userNr]
#     selected_ind = selected_ind[beats_skiped:-beats_skiped]
#     train_ind_lst.extend(selected_ind.to_list())

# # Anotacijų pasiskirstymas mokymo duomenyse
# print("\nAnotacijų pasiskirstymas mokymo duomenyse\n")
# labels_table, labels_sums = anotaciju_pasiskirstymas_v2(all_beats_attr, ind_lst=train_ind_lst, cols_pattern=['N', 'S', 'V'])
# print(labels_table)
# print("\nTotal:", labels_sums['All'])

# # Įrašome mokymo imties sekų indeksų sąrašą į diską
# file_path = Path(SCHEMA_PATH, 'train_ind_lst.csv')
# np.savetxt(file_path, np.array(train_ind_lst), delimiter=',', fmt='%d')
# print("\nMokymo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_train, y_train = create_set_from_rec(rec_dir, all_beats_attr, train_ind_lst, window_left_side, window_right_side)
# print("\nMokymo imties dimensijos")
# print(f"{X_train.shape=} {y_train.shape=}")

# Įrašome mokymui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'train_set.npz')
# np.savez_compressed(file_path, X=X_train, y=y_train)
# print("\nMokymo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )


# VERTINIMO IMTIS
# Suformuojame vertinimo imties sekų indeksų sąrašą. Formuodami sąrašą
# kiekvienam pacientui eliminuojame pirmus ir paskutinius beats_skiped indeksus

validate_ind_lst = []
for SubjCode in DS2:
    userNr, recordingNr = split_SubjCode(SubjCode)
    selected_ind = all_beat_indices[(all_beats_attr['userNr']==userNr) & (all_beats_attr['recordingNr']==recordingNr)]
    selected_ind = selected_ind[beats_skiped:-beats_skiped]
    selected_ind_lst = selected_ind.to_list()
    print("selected_ind_lst parameters:")
    print(f"SubjCode: {SubjCode}  first elem: {selected_ind_lst[0]} last elem: {selected_ind_lst[-1]}  tot: {len(selected_ind_lst)}")
    validate_ind_lst.extend(selected_ind_lst)

# Anotacijų pasiskirstymas vertinimo duomenyse
print("\nAnotacijų pasiskirstymas vertinimo duomenyse\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr, ind_lst= validate_ind_lst, cols_pattern=cols_pattern)
print_annotations_table(labels_table, labels_sums, Flag1=False, Flag2=False)

# Įrašome vertinimo imties sekų indeksų sąrašą į diską
file_path = Path(SCHEMA_PATH, 'validate_ind_lst.csv')
np.savetxt(file_path, np.array(validate_ind_lst), delimiter=',', fmt='%d')
print("\nVertinimo imties sekų indeksų sąrašas įrašytas į: ", file_path )

# Suformuojame mokymo sekų ir jų klasių numerių masyvus
# X_validate, y_validate = create_set_from_rec(rec_dir, all_beats_attr, validate_ind_lst, window_left_side, window_right_side)
# print("\nVertinimo imties dimensijos")
# print(f"{X_validate.shape=} {y_validate.shape=}")

# Įrašome vertinimui skirtus masyvus į npz file diske
# file_path = Path(SCHEMA_PATH, 'validate_set.npz')
# np.savez_compressed(file_path, X=X_validate, y=y_validate)
# print("\nVertinimo sekų ir jų klasių numerių masyvas įrašytas: į ", file_path )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

# Suformuojame info_prep masyvą ir įrašome į diską

info = {
    'fs': fs,
    'SubjCodes':SubjCodes,
    'selected_beats':selected_beats,
    'beats_skiped':beats_skiped
    }

file_path = Path(SCHEMA_PATH,'info_prep_tst.json')
with open(file_path, 'w') as f:
    json.dump(info, f)
    
print("\nSchemos parametrai info.json yra aplanke: ", file_path, "\n" )

print("\nPabaiga.............")



Anotacijų pasiskirstymas visuose duomenyse

'N': 215506, 'S': 2438, 'V': 6999
Total:  224943
selected_ind_lst parameters:
SubjCode: 10000  first elem: 10 last elem: 5744  tot: 5735
selected_ind_lst parameters:
SubjCode: 10001  first elem: 5765 last elem: 6605  tot: 841
selected_ind_lst parameters:
SubjCode: 10002  first elem: 6626 last elem: 7371  tot: 746
selected_ind_lst parameters:
SubjCode: 10003  first elem: 7392 last elem: 12221  tot: 4830
selected_ind_lst parameters:
SubjCode: 10010  first elem: 12242 last elem: 13086  tot: 845
selected_ind_lst parameters:
SubjCode: 10020  first elem: 13107 last elem: 19104  tot: 5998
selected_ind_lst parameters:
SubjCode: 10021  first elem: 19125 last elem: 19897  tot: 773
selected_ind_lst parameters:
SubjCode: 10022  first elem: 19918 last elem: 20704  tot: 787
selected_ind_lst parameters:
SubjCode: 10023  first elem: 20725 last elem: 21467  tot: 743
selected_ind_lst parameters:
SubjCode: 10024  first elem: 21488 last elem: 22336  tot: 849
se